# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
api_key = 'FIRBuNuP6Au1k53Ys4MupsZaWWE6baB8GMCFNGUOTjSj4-u3A-mJh76nkxPlcyv3dDAYENVtUesOOS6oLwC09n3g0nte42DQf3e0sxgf7YNMmL3d5jQzqKUI4jQSXXYx'


In [2]:
#Your code here
import requests

term = 'Healthy'
location = 'NYC'
search_limit = 50

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term,
                'location': location,
                'limit': search_limit
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)

<Response [200]>


## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [3]:
len(response.json()['businesses'])

50

In [4]:
response.json()['total']

1900

In [6]:
import time

term = 'Healthy'
location = 'NYC'
search_limit = 50

url_params = {
                'term': term,
                'location': location,
                'limit': search_limit,
                'offset': 50
            }

count = 0
business_list = []
while count < (100/50):
    request = requests.get(url, headers=headers, params=url_params)
    if request.status_code == 200:
        business_list += response.json()['businesses']
        count += 1
        time.sleep(0.1)
    else:
        print('Unexpected Error from Server')
        break

In [7]:
print(len(business_list))
business_list[:5]

100


[{'id': 'yH49flYj7qk9JOX124uhFg',
  'alias': 'mthr-vegan-new-york-2',
  'name': 'MTHR Vegan',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/7NOnN1MqUeJMxdEvnTiMMw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/mthr-vegan-new-york-2?adjust_creative=mz_sq35JM_TZ3xUfYUT14A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mz_sq35JM_TZ3xUfYUT14A',
  'review_count': 62,
  'categories': [{'alias': 'vegan', 'title': 'Vegan'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.76241, 'longitude': -73.98806},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '329 W 49th St',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10019',
   'country': 'US',
   'state': 'NY',
   'display_address': ['329 W 49th St', 'New York, NY 10019']},
  'phone': '+12127578600',
  'display_phone': '(212) 757-8600',
  'distance': 6353.551891409855},
 {'id': 'XipQLDbyTl5tsLlyzAWzug',
  'alias': 'jajaja-new

In [9]:
import pandas as pd

df_business = pd.DataFrame.from_dict(business_list)

In [10]:
# Your code here; use a function or loop to retrieve all the results from your original request
df_business.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,mthr-vegan-new-york-2,"[{'alias': 'vegan', 'title': 'Vegan'}]","{'latitude': 40.76241, 'longitude': -73.98806}",(212) 757-8600,6353.551891,yH49flYj7qk9JOX124uhFg,https://s3-media2.fl.yelpcdn.com/bphoto/7NOnN1...,False,"{'address1': '329 W 49th St', 'address2': None...",MTHR Vegan,+12127578600,$$,4.5,62,"[pickup, delivery]",https://www.yelp.com/biz/mthr-vegan-new-york-2...
1,jajaja-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.7141530317142, 'longitude': -7...",(646) 883-5453,1021.727407,XipQLDbyTl5tsLlyzAWzug,https://s3-media2.fl.yelpcdn.com/bphoto/QaIhCE...,False,"{'address1': '162 E Broadway', 'address2': '',...",Jajaja,+16468835453,$$,4.5,706,"[pickup, delivery, restaurant_reservation]",https://www.yelp.com/biz/jajaja-new-york?adjus...
2,le-botaniste-new-york-4,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...","{'latitude': 40.7204278, 'longitude': -74.0002...",(646) 870-7770,1746.485460,4b43TOXpskBPZqQzbKkwDQ,https://s3-media4.fl.yelpcdn.com/bphoto/UxNilz...,False,"{'address1': '127 Grand St', 'address2': None,...",Le Botaniste,+16468707770,$$,4.5,107,[],https://www.yelp.com/biz/le-botaniste-new-york...
3,avocaderia-brooklyn,"[{'alias': 'juicebars', 'title': 'Juice Bars &...","{'latitude': 40.65602, 'longitude': -74.00742}",(347) 227-0350,5595.057473,dqfHeckcMJLhrRSjlPIynw,https://s3-media3.fl.yelpcdn.com/bphoto/3lnP1E...,False,"{'address1': '254 36th St', 'address2': 'Bldg ...",Avocaderia,+13472270350,$$,3.5,217,[],https://www.yelp.com/biz/avocaderia-brooklyn?a...
4,the-shanti-shack-brooklyn,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...","{'latitude': 40.7171867, 'longitude': -73.9623...",(347) 463-9886,2992.938919,LcM9IizN90KGc3gYsW1_Hw,https://s3-media1.fl.yelpcdn.com/bphoto/y5kBlX...,False,"{'address1': '85 N 3rd St', 'address2': '', 'a...",The Shanti Shack,+13474639886,$$,5.0,29,[],https://www.yelp.com/biz/the-shanti-shack-broo...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [16]:
#Your code here
sep_coord = df_business.coordinates.apply(pd.Series)
sep_df = pd.concat([df_business, sep_coord], axis=1).drop('coordinates', axis=1)
sep_df.head()

,alias,categories,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,latitude,longitude
0,mthr-vegan-new-york-2,"[{'alias': 'vegan', 'title': 'Vegan'}]",(212) 757-8600,6353.551891,yH49flYj7qk9JOX124uhFg,https://s3-media2.fl.yelpcdn.com/bphoto/7NOnN1...,False,"{'address1': '329 W 49th St', 'address2': None...",MTHR Vegan,+12127578600,$$,4.5,62,"[pickup, delivery]",https://www.yelp.com/biz/mthr-vegan-new-york-2...,40.762410,-73.988060
1,jajaja-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",(646) 883-5453,1021.727407,XipQLDbyTl5tsLlyzAWzug,https://s3-media2.fl.yelpcdn.com/bphoto/QaIhCE...,False,"{'address1': '162 E Broadway', 'address2': '',...",Jajaja,+16468835453,$$,4.5,706,"[pickup, delivery, restaurant_reservation]",https://www.yelp.com/biz/jajaja-new-york?adjus...,40.714153,-73.990425
2,le-botaniste-new-york-4,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",(646) 870-7770,1746.485460,4b43TOXpskBPZqQzbKkwDQ,https://s3-media4.fl.yelpcdn.com/bphoto/UxNilz...,False,"{'address1': '127 Grand St', 'address2': None,...",Le Botaniste,+16468707770,$$,4.5,107,[],https://www.yelp.com/biz/le-botaniste-new-york...,40.720428,-74.000203
3,avocaderia-brooklyn,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",(347) 227-0350,5595.057473,dqfHeckcMJLhrRSjlPIynw,https://s3-media3.fl.yelpcdn.com/bphoto/3lnP1E...,False,"{'address1': '254 36th St', 'address2': 'Bldg ...",Avocaderia,+13472270350,$$,3.5,217,[],https://www.yelp.com/biz/avocaderia-brooklyn?a...,40.656020,-74.007420
4,the-shanti-shack-brooklyn,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",(347) 463-9886,2992.938919,LcM9IizN90KGc3gYsW1_Hw,https://s3-media1.fl.yelpcdn.com/bphoto/y5kBlX...,False,"{'address1': '85 N 3rd St', 'address2': '', 'a...",The Shanti Shack,+13474639886,$$,5.0,29,[],https://www.yelp.com/biz/the-shanti-shack-broo...,40.717187,-73.962341


In [20]:
import folium

base_map = folium.Map([sep_df.latitude.iloc[0], sep_df.longitude.iloc[0]], zoom_start=13)

for index in range(len(sep_df)):
    lat = sep_df.latitude.iloc[index]
    long = sep_df.longitude.iloc[index]
    popup_text = "Latitude: {}, Longitude: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)

base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!